In [18]:
!pip install torch

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2


In [1]:
!pip install accelerate~=0.20
!pip install transformers~=4.30.2


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import json
import pandas as pd
pd.options.mode.chained_assignment = None

In [7]:
df = pd.read_csv("/root/trial-mt/Data/Raw_Data/bitcoin/submission_bitcoin.csv")
df

,submission,subreddit,author,created,retrieved,edited,pinned,archived,locked,removed,...,link_flair_text,upvote_ratio,score,gilded,total_awards_received,num_comments,num_crossposts,selftext,thumbnail,shortlink
0,rt6r1a,bitcoin,boom888boom,1640995385,1641140399,0,0,0,0,1,...,off topic,1.00,3,0,0,2,0,[removed],default,https://redd.it/rt6r1a
1,rt6s86,bitcoin,[deleted],1640995487,1641140399,0,0,0,1,1,...,altcoin,0.43,0,0,0,2,0,[removed],default,https://redd.it/rt6s86
2,rt6tub,bitcoin,FSdotorg,1640995629,1641140396,0,0,0,1,1,...,altcoin,0.33,0,0,0,2,0,[removed],default,https://redd.it/rt6tub
3,rt711v,bitcoin,Squigglywiggler,1640996258,1641140399,0,0,0,0,0,...,NaN,0.75,12,0,0,9,0,Anyone heard any news about this? Any opinions...,self,https://redd.it/rt711v
4,rt76mj,bitcoin,BuyTheDipper,1640996745,1641140399,0,0,0,0,1,...,repetitive,0.75,2,0,0,4,0,[removed],default,https://redd.it/rt76mj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66571,1005i5m,bitcoin,bamboochar,1672526183,1672536965,1672526489,0,0,0,0,...,NaN,0.85,5,0,0,4,0,I am in the process of setting up a multisig s...,self,https://redd.it/1005i5m
66572,1005rx4,bitcoin,Eddie_investor,1672527004,1672536965,0,0,0,0,1,...,off topic,0.53,1,0,0,15,0,[removed],https://b.thumbs.redditmedia.com/yWemvJ_EzuawV...,https://redd.it/1005rx4
66573,1006hhl,bitcoin,Drunktraveler99,1672529206,1672536965,0,0,0,0,0,...,NaN,0.70,4,0,0,3,0,"Been holding bitcoin for about 3 years, today ...",self,https://redd.it/1006hhl
66574,1006m5m,bitcoin,CommunicationAway341,1672529618,1672536965,0,0,0,0,0,...,NaN,0.81,15,0,0,4,0,NaN,https://b.thumbs.redditmedia.com/8FCDqbwC9VQdP...,https://redd.it/1006m5m


In [8]:
df.columns

Index(['submission', 'subreddit', 'author', 'created', 'retrieved', 'edited',
       'pinned', 'archived', 'locked', 'removed', 'deleted', 'is_self',
       'is_video', 'is_original_content', 'title', 'link_flair_text',
       'upvote_ratio', 'score', 'gilded', 'total_awards_received',
       'num_comments', 'num_crossposts', 'selftext', 'thumbnail', 'shortlink'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66576 entries, 0 to 66575
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   submission             66576 non-null  object 
 1   subreddit              66576 non-null  object 
 2   author                 66576 non-null  object 
 3   created                66576 non-null  int64  
 4   retrieved              66576 non-null  int64  
 5   edited                 66576 non-null  int64  
 6   pinned                 66576 non-null  int64  
 7   archived               66576 non-null  int64  
 8   locked                 66576 non-null  int64  
 9   removed                66576 non-null  int64  
 10  deleted                66576 non-null  int64  
 11  is_self                66576 non-null  int64  
 12  is_video               66576 non-null  int64  
 13  is_original_content    66576 non-null  int64  
 14  title                  66567 non-null  object 
 15  li

In [10]:
# Extracting only reqd columns
df = df[['submission','author','created','num_comments','shortlink','link_flair_text','title','selftext','score','subreddit']]

In [11]:
df['posted_on'] = pd.to_datetime(df['created'], unit ='s')

In [12]:
df.drop('created',axis=1, inplace=True)

In [13]:
df

,submission,author,num_comments,shortlink,link_flair_text,title,selftext,score,subreddit,posted_on
0,rt6r1a,boom888boom,2,https://redd.it/rt6r1a,off topic,Anyone use a roundup app to invest spare change?,[removed],3,bitcoin,2022-01-01 00:03:05
1,rt6s86,[deleted],2,https://redd.it/rt6s86,altcoin,"Maybe I am more pessimistic, so I withdrew Bit...",[removed],0,bitcoin,2022-01-01 00:04:47
2,rt6tub,FSdotorg,2,https://redd.it/rt6tub,altcoin,Can not withdraw,[removed],0,bitcoin,2022-01-01 00:07:09
3,rt711v,Squigglywiggler,9,https://redd.it/rt711v,NaN,When sovereign wealth funds?,Anyone heard any news about this? Any opinions...,12,bitcoin,2022-01-01 00:17:38
4,rt76mj,BuyTheDipper,4,https://redd.it/rt76mj,repetitive,Big Money Lost in a Hard Drive,[removed],2,bitcoin,2022-01-01 00:25:45
...,...,...,...,...,...,...,...,...,...,...
66571,1005i5m,bamboochar,4,https://redd.it/1005i5m,NaN,Multisig - multiple uses for each pkey?,I am in the process of setting up a multisig s...,5,bitcoin,2022-12-31 22:36:23
66572,1005rx4,Eddie_investor,15,https://redd.it/1005rx4,off topic,Really ???,[removed],1,bitcoin,2022-12-31 22:50:04
66573,1006hhl,Drunktraveler99,3,https://redd.it/1006hhl,NaN,Bitcoin Lake Guatemala,"Been holding bitcoin for about 3 years, today ...",4,bitcoin,2022-12-31 23:26:46
66574,1006m5m,CommunicationAway341,4,https://redd.it/1006m5m,NaN,Start the new year of right.,NaN,15,bitcoin,2022-12-31 23:33:38


In [14]:
# Extracting posts only if their body/content length >=20
df = df[df ['selftext'].str.len() >= 20]

In [15]:
df.sample(20)

,submission,author,num_comments,shortlink,link_flair_text,title,selftext,score,subreddit,posted_on
52345,xk1zln,Exact_Combination_38,67,https://redd.it/xk1zln,NaN,How to be in the top 1% long-term,"So, I've done the math. Not because it's hard ...",86,bitcoin,2022-09-21 11:14:32
2603,s18x4y,TheDumbInvesto,9,https://redd.it/s18x4y,NaN,Which one is a better evil?,Hi - if someone is not in a position to take s...,0,bitcoin,2022-01-11 09:18:36
12357,sv7rgn,Icy-Wrongdoer7778,8,https://redd.it/sv7rgn,NaN,I met the first man to ever use a BTC atm,"I'm not trolling, I'm completely serious. He's...",0,bitcoin,2022-02-18 03:31:18
55906,y8jtl2,mintspit,11,https://redd.it/y8jtl2,NaN,Question about cold wallets,How anonymous is let’s say a Ledger cold walle...,6,bitcoin,2022-10-20 01:00:05
46671,wjy610,Solid_Competition354,37,https://redd.it/wjy610,NaN,Need help with cold storage,I want to start accumulating btc in my cold wa...,12,bitcoin,2022-08-09 08:46:58
9126,skri0v,djtetsu,1,https://redd.it/skri0v,NaN,Exchanges with new address creation,"As the title says, I'm looking for exchanges t...",1,bitcoin,2022-02-04 23:16:59
15218,t4o5jj,serunomismo,1,https://redd.it/t4o5jj,NaN,"Current $BTC 🔰 Holders: 40,541,367 📈 ⬆️",Always Up 🆙\n\nSource👇\n\n[Bitcoin Rich Addres...,4,bitcoin,2022-03-02 01:31:12
26845,uhwfms,Iguana_The_Wise,5,https://redd.it/uhwfms,NaN,Banks assign you account numbers. With Bitcoin...,That's right.\n\nYour wallet creates these add...,30,bitcoin,2022-05-04 02:27:19
2369,s0kcgx,SnooCheesecakes9425,2,https://redd.it/s0kcgx,NaN,I will do random tasks for a Bitcoin:,"Hi all, I will do random tasks for a Bitcoin; ...",0,bitcoin,2022-01-10 13:44:39
23956,u4kny9,ShotBot,55,https://redd.it/u4kny9,NaN,What Raoul Pal gets wrong about Bitcoin:,Raoul Pal sold his coins because he believes o...,59,bitcoin,2022-04-15 23:28:01


In [16]:
df.shape

(16058, 10)

In [17]:
# Removing Daily Discussion posts
print(sum(df['title'].str.contains('Daily Discussion')))
final_df = df[~df['title'].str.contains('Daily Discussion')]

365


In [18]:
# Resetting Index to be in sequence
final_df.reset_index(drop=True, inplace=True)

## Sentiment Analysis

In [3]:
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=64, truncation=True, padding = 'max_length')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [19]:
reddit_texts = final_df['selftext'].tolist()

In [20]:
preds = pipe(reddit_texts)
preds

[{'label': 'Neutral', 'score': 0.6690639853477478},
 {'label': 'Bullish', 'score': 0.6554852724075317},
 {'label': 'Bullish', 'score': 0.6189212799072266},
 {'label': 'Neutral', 'score': 0.8952701687812805},
 {'label': 'Neutral', 'score': 0.5783287882804871},
 {'label': 'Bullish', 'score': 0.5919817686080933},
 {'label': 'Bullish', 'score': 0.5707654356956482},
 {'label': 'Bullish', 'score': 0.6008257865905762},
 {'label': 'Neutral', 'score': 0.739990234375},
 {'label': 'Neutral', 'score': 0.7487289309501648},
 {'label': 'Bullish', 'score': 0.5627982020378113},
 {'label': 'Bullish', 'score': 0.553986132144928},
 {'label': 'Neutral', 'score': 0.667134702205658},
 {'label': 'Bullish', 'score': 0.7251851558685303},
 {'label': 'Bullish', 'score': 0.5050667524337769},
 {'label': 'Neutral', 'score': 0.6311385631561279},
 {'label': 'Bearish', 'score': 0.36945077776908875},
 {'label': 'Neutral', 'score': 0.796308159828186},
 {'label': 'Bullish', 'score': 0.642392635345459},
 {'label': 'Neutral

In [26]:
# lst_labels = {}
for element in preds:
    label = element['label']
    if label == "Bearish":
        element['label']="Negative"
    elif label == 'Bullish':
        element['label']="Positive"
    else:
        element['label']="Neutral"

In [28]:
new_data_df = pd.DataFrame(preds)

In [29]:
new_data_df

,label,score
0,Neutral,0.669064
1,Positive,0.655485
2,Positive,0.618921
3,Neutral,0.895270
4,Neutral,0.578329
...,...,...
15688,Neutral,0.806706
15689,Neutral,0.522978
15690,Neutral,0.583011
15691,Positive,0.689596


In [30]:
print(new_data_df.score.min(),new_data_df.score.max())

0.3355315029621124 0.9943162798881531


In [32]:
# first_100 = final_df[:100]
# final_first_100 = pd.concat([final_df, new_data_df], axis=1)
labeled_df = pd.concat([final_df, new_data_df], axis=1)

In [33]:
labeled_df.to_csv("/root/trial-mt/Data/Sentiment/bitcoin_submission_sentiment.csv",index=False)

In [35]:
sum(labeled_df['label'] == 'Neutral')

8942

In [36]:
sum(labeled_df['label'] == 'Positive')

5651

In [37]:
sum(labeled_df['label'] == 'Negative')

1100